# Computational assignment 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate

from nodepy import runge_kutta_method as rk, ivp

In [ ]:
from matplotlib import rc
# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (15, 6),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,}
plt.rcParams.update(newparams)

In [ ]:
# Setup
m = 1 #kg
R = 1 #m
g = 9.81 #m/s^2

theta_0 = 0.2 # rad, inital angle
omega_0 = 0.0 # rad/s, inital angular velocity

end_time = 5 # s, the time at which the simulation will end
#dt = 1e-2 # s, step size used in Euler's method

The problem consists of two first-order differential equations:
$$
\dot{\theta} = \omega\\
\dot{\omega} = \frac{F_{\perp}}{mR}.
$$

Let 
$$
y = 
\begin{bmatrix}
\theta\\
\omega
\end{bmatrix},
$$
then
$$
y' = 
\begin{bmatrix}
y_2\\
-\sin(y_1) / mR
\end{bmatrix}
= f(y).
$$

We will do the euler step:
$$
y(t+dt) = y(t) + f(y(t)) dt
$$

In [ ]:
def f(y):
    """f(y) = y'
    """
    return np.array([y[1], -g*np.sin(y[0])/R])

def f_time(t, y):
    """Wrapper for f(y) to comply with standardt call-signature in scipy"""
    return f(y)

In [ ]:
def euler_step(y, f, dt):
    """Returns an euler-approximation of y(t+dt) given y(t)"""
    return y + f(y)*dt

In [ ]:
def euler_solve(y, f, dt, num_iterations):
    """Solve using Euler's method"""
    for i in range(num_iterations - 1):
        y[i+1] = euler_step(y[i], f, dt)
    return y

def euler_cromer_solve(y, f, dt, num_iterations):
    """Solve using Euler-Cromer's method.
    NB, this is not the most efficient impementation,
    it is lazy, but simple as it reuses existing code"""
    for i in range(num_iterations - 1):
        theta, omega = euler_step(y[i], f, dt)
        theta = y[i,0] + omega*dt
        y[i+1] = np.array([theta, omega])
    return y

In [ ]:
def kinetic_energy(omega):
    return 0.5*m*R**2*omega**2

def potential_energy(theta):
    return m*g*R*(1-np.cos(theta))

def total_energy(y1, y2):
    return kinetic_energy(y2) + potential_energy(y1)

def plt_method_energy(t, theta, omega, name):
    plt.plot(t, potential_energy(theta), label=f"{name} potential")
    plt.plot(t, kinetic_energy(theta), label=f"{name} kinetic")
    plt.plot(t, total_energy(theta, omega), label=f"{name} total")

## Part one, Euler method for different time-delta

In [ ]:
an = lambda t: theta_0*np.cos(np.sqrt(g/R)*t)

In [ ]:
# Using Euler's method

def do_method(end_time, dt, solver=euler_solve, theta_0=theta_0, omega_0=omega_0):
    """Use method given end_time, dt
    Returns:
     t: array of times
     y: 2D-array with [theta, omega]"""
    num_iterations = int(np.ceil(end_time/dt))
    y = np.zeros((num_iterations, 2))
    y[0] = np.array([theta_0, omega_0])
    y = solver(y, f, dt, num_iterations)
    t = np.arange(0, end_time, dt)
    
    return t, y

dt_list = [1e-4, 1e-3, 1e-2, 5e-2]
y_euler_dt = {}
for dt in dt_list:
    y_euler_dt[dt] = do_method(end_time, dt)

In [ ]:
for i in range(len(dt_list)):
    plt.subplot(f"{len(dt_list)}2{2*i+1}")
    dt = dt_list[i]
    t = y_euler_dt[dt][0]
    y = y_euler_dt[dt][1]
    plt.title(f"dt = {dt}")
    plt.plot(t, y[:,0], label="Theta")
    t_n = np.linspace(0, end_time, 100)
    plt.plot(t_n, an(t_n), label="Analytical")
    plt.legend()
    
    plt.subplot(f"{len(dt_list)}2{2*i+2}")
    plt_method_energy(t, y[:,0], y[:,1], "Euler")
    
plt.tight_layout()

In [ ]:
# Using Euler-Cromer's method
t, y = do_method(end_time, 0.01, euler_cromer_solve)

plt.plot(t, y[:,0])

In [ ]:
# Using RK45
sol_RK45 = scipy.integrate.solve_ivp(f_time, [0.0, end_time*100], [theta_0, omega_0])
y_rk45 = sol_RK45.y
t_rk45 = sol_RK45.t

In [ ]:
plt_method_energy(t_rk45, y_rk45[0], y_rk45[1], "RK45")

In [ ]:
plt.plot(t_rk45, y_rk45[0])

## Let's do some plotting!!
First is $\omega(t)$ for all methods

In [ ]:
plt.title("$\\theta(t)$")
plt.plot(t_euler, y_euler[:,0], label="Euler")
plt.plot(t_euler, y_euler_cromer[:,0], label="Euler-Cromer")
plt.plot(t_rk45, y_rk45[0], label="RK45")
plt.xlabel("t [s]")
plt.ylabel("$\\theta(t)$ [rad]")
plt.legend()
plt.show()

In [ ]:


plt.title("$~v^2$")
plt.subplot("311")
plt_method_energy(t_euler, y_euler[:,0], y_euler[:,1], "Euler")
plt.subplot("312")
plt_method_energy(t_euler, y_euler_cromer[:,0], y_euler_cromer[:,1], "Euler-Cromer")
plt.subplot("313")
plt_method_energy(t_rk45, y_rk45[0], y_rk45[1], "RK45")

plt.show()

It is interesting to look at the time-development of $v^2 \propto T$ as this sould be time-constant.

Plotted in phase space:

In [ ]:
plt.plot(y_euler[:,0], y_euler[:,1], label="Euler")
plt.plot(y_euler_cromer[:,0], y_euler_cromer[:,1], label="Euler-Cromer")
plt.plot(y_rk45[0], y_rk45[1], label="RK45")
plt.xlabel("$\\theta$")
plt.ylabel("$\omega$")
plt.legend()
plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set up the figure
fig = plt.figure(figsize=(4, 4), dpi=60)
ax = plt.axes(xlim=(-1.1*R, 1.1*R), ylim=(-1.1*R, 0.1*R))
ax.set_aspect('equal')

pend, = ax.plot([0, np.sin(y_euler[0,0])*R], [0, -np.cos(y_euler[0,0])*R])
ball, = ax.plot([np.sin(y_euler[0,0])*R], [-np.cos(y_euler[0,0])*R], 'o', markersize=10)

def animate(y):
    pend.set_data([0, np.sin(y)*R], [0, -np.cos(y)*R])
    ball.set_data([np.sin(y)*R], [-np.cos(y)*R])
    
anim = FuncAnimation(fig, animate, interval=1000*dt, frames=y_euler[:,0])


plt.close(anim._fig)
HTML(anim.to_html5_video())